# populate database with defaults

> Populate the created database with the default values for 'modifiers', 'actions', 'games', 'game_keys', 'categories' and 'bindings'. 

In [ ]:
#| default_exp populate_db_defaults

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from fasthtml.common import *
from keybindings_fps.create_db_structure import *
from keybindings_fps.manipulate_db_contents import *

In [ ]:
#|export
def populate_categories(db):
    """Populate the categories table with initial data"""
    categories = [
        ('movement', 'Movement related actions'),
        ('combat', 'Combat related actions'),
        ('interaction', 'Manipulate the environment'),
        ('communication', 'Communicate with friends and bots'),
        ('menu', 'Access the menus')
    ]
    
    for name, description in categories:
        db.t.categories.insert(dict(name=name, description=description))


In [ ]:
#|export
def populate_modifiers(db):
    """Populate the modifiers table with initial data"""
    modifiers = ['tap', 'hold', 'scroll', 'double_tap']
    
    for name in modifiers:
        db.t.modifiers.insert(dict(name=name))

In [ ]:
#|export
def populate_game_keys(db):
    """Populate the game_keys table with initial data"""
    # Letters
    letters = [chr(i) for i in range(ord('a'), ord('z')+1)]
    
    # Numbers
    numbers = [str(i) for i in range(10)]
    
    # Function keys
    function_keys = [f'f{i}' for i in range(1, 13)]
    
    # Special keys
    special_keys = ['left_alt', 'enter', 'space', 'left_control', 'tab', '.', 
                   'left_shift', 'esc', 'del', ';', '-', '`', '=', "'",
                   'mouse_left', 'mouse_right', 'mouse_side_front', 
                   'mouse_side_back', 'mouse_middle', 'mouse_sniper']
    
    # Add all keys
    for key in letters + numbers + function_keys + special_keys:
        db.t.game_keys.insert(dict(name=key))

In [ ]:
#|export
def populate_actions(db):
    """Populate the actions table with initial data"""
    # Get category IDs for reference
    categories = {c['name']: c['id'] for c in db.t.categories()}
    
    actions_by_category = {
        'movement': [
            'Forward', 'Backward', 'Left', 'Right', 'Jump/climb', 
            'Crouch', 'Prone', 'Sprint', 'Walk', 'Lean left', 'Lean right'
        ],
        'combat': [
            'Fire primary', 'Aim', 'Reload', 'Switch weapon', 
            'Melee', 'Grenade', 'Special', 'Stim'
        ],
        'interaction': ['Use'],
        'communication': ['Voice', 'Text', 'Ping', 'Emote'],
        'menu': ['Menu', 'Map', 'Inventory', 'Special']
    }
    
    # Add all actions with their category IDs
    for category, actions in actions_by_category.items():
        for action in actions:
            db.t.actions.insert(dict(
                name=action,
                category_id=categories[category]
            ))


In [ ]:
#|export
def create_default_game(db):
    """Create the default game template with standard bindings"""
    # Create default game
    db.t.games.insert(dict(
        name='default',
        game_type='template',
        image=None
    ))

    # Default bindings dictionary with category-based sort_order
    default_bindings = {
        # Movement (100-199)
        'Forward': ('w', 'hold', None, 101),
        'Backward': ('s', 'hold', None, 102),
        'Left': ('a', 'hold', None, 103),
        'Right': ('d', 'hold', None, 104),
        'Jump/climb': ('left_control', 'tap', None, 105),
        'Crouch': ('c', 'tap', None, 106),
        'Prone': ('space', 'tap', None, 107),
        'Sprint': ('left_shift', 'hold', None, 108),
        'Walk': ('left_alt', 'hold', None, 109),
        'Lean left': ('mouse_side_front', 'hold', None, 110),
        'Lean right': ('mouse_side_back', 'hold', None, 111),
        
        # Combat (200-299)
        'Fire primary': ('mouse_left', 'tap', None, 201),
        'Aim': ('mouse_right', 'tap', None, 202),
        'Reload': ('r', 'tap', None, 203),
        'Switch weapon': ('mouse_middle', 'scroll', None, 204),
        'Melee': ('h', 'tap', None, 205),
        'Grenade': ('g', 'tap', None, 206),
        'Special': ('q', 'tap', None, 207),
        'Stim': ('f', 'tap', None, 208),
        
        # Interaction (300-399)
        'Use': ('e', 'tap', None, 301),
        
        # Communication (400-499)
        'Voice': ('t', 'tap', None, 401),
        'Text': ('enter', 'tap', None, 402),
        'Ping': ('z', 'tap', None, 403),
        'Emote': ('b', 'tap', None, 404),
        
        # Menu (500-599)
        'Menu': ('esc', 'tap', None, 501),
        'Map': ('m', 'tap', None, 502),
        'Inventory': ('tab', 'tap', None, 503)
    }

    # Add all bindings using existing add_binding function
    for action, (key, modifier, description, sort_order) in default_bindings.items():
        add_binding(db, 'default', action, key, modifier, description, sort_order)

In [ ]:
#| export
def setup_fresh_db():
    """Create a fresh database with all initial data"""
    db = init_db()
    create_tables(db)
    populate_categories(db)
    populate_modifiers(db)
    populate_game_keys(db)
    populate_actions(db)
    create_default_game(db)
    return db

## Start with a fresh database

WARNING: This will erase all data in the database!

In [ ]:
#| export
db = setup_fresh_db()
#db.t.categories()

## Use an existing database on disk

In [ ]:
db = init_db()
db.t.categories()

[{'id': 1, 'name': 'movement', 'description': 'Movement related actions'},
 {'id': 2, 'name': 'combat', 'description': 'Combat related actions'},
 {'id': 3, 'name': 'interaction', 'description': 'Manipulate the environment'},
 {'id': 4,
  'name': 'communication',
  'description': 'Communicate with friends and bots'},
 {'id': 5, 'name': 'menu', 'description': 'Access the menus'}]

In [ ]:
db.t.bindings()

[{'id': 1,
  'game_id': 1,
  'action_id': 1,
  'key_id': 23,
  'modifier_id': 2,
  'description': None,
  'sort_order': 101},
 {'id': 2,
  'game_id': 1,
  'action_id': 2,
  'key_id': 19,
  'modifier_id': 2,
  'description': None,
  'sort_order': 102},
 {'id': 3,
  'game_id': 1,
  'action_id': 3,
  'key_id': 1,
  'modifier_id': 2,
  'description': None,
  'sort_order': 103},
 {'id': 4,
  'game_id': 1,
  'action_id': 4,
  'key_id': 4,
  'modifier_id': 2,
  'description': None,
  'sort_order': 104},
 {'id': 5,
  'game_id': 1,
  'action_id': 5,
  'key_id': 52,
  'modifier_id': 1,
  'description': None,
  'sort_order': 105},
 {'id': 6,
  'game_id': 1,
  'action_id': 6,
  'key_id': 3,
  'modifier_id': 1,
  'description': None,
  'sort_order': 106},
 {'id': 7,
  'game_id': 1,
  'action_id': 7,
  'key_id': 51,
  'modifier_id': 1,
  'description': None,
  'sort_order': 107},
 {'id': 8,
  'game_id': 1,
  'action_id': 8,
  'key_id': 55,
  'modifier_id': 2,
  'description': None,
  'sort_order': 1

In [ ]:
game_id=1
db.t.bindings.rows_where("game_id = ?", game_id)

<generator object Queryable.rows_where>

In [ ]:
db.t

actions, bindings, categories, game_keys, games, modifiers, sqlite_stat1, sqlite_stat4

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()